<a href="https://colab.research.google.com/github/VivekVSH01/Python/blob/main/PP_Bot_State.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive to save files persistently
from google.colab import drive
drive.mount('/content/drive')

# Install required packages
!pip install sentence-transformers faiss-cpu bitsandbytes transformers peft

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.9 MB/s eta 0:00:00
  Attempting uninsta

In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())

CUDA available: False


In [ ]:
pip install nltk

In [ ]:
import requests, re, os, textwrap, pathlib

url = "https://www.gutenberg.org/files/1342/1342-0.txt"
raw = requests.get(url, timeout=30).text

clean = re.sub(r"(?s)\*\*\* START.*?END \*\*\*", "", raw)
sentences = re.split(r"(?<=[.!?])\s+(?=[A-Z])", clean)

chunks, cur, cur_len, LIM = [], "", 0, 512
for s in sentences:
    tok = s.split()
    if cur_len + len(tok) > LIM:
        chunks.append(cur.strip()); cur, cur_len = "", 0
    cur += s + " "; cur_len += len(tok)
if cur.strip(): chunks.append(cur.strip())

outf = pathlib.Path("/content/drive/MyDrive/gutenberg_chunks.txt")
outf.parent.mkdir(parents=True, exist_ok=True)
outf.write_text("\n\n".join(chunks))
print(f"Total chunks: {len(chunks)}")

Total chunks: 261


In [ ]:
import faiss

In [ ]:
import spacy, re, os, numpy as np
from pathlib import Path

chunk_path = Path("/content/drive/MyDrive/gutenberg_chunks.npy")
if chunk_path.exists():
    chunks = np.load(chunk_path, allow_pickle=True).tolist()
else:
    import requests
    nlp = spacy.blank("en")
    nlp.add_pipe("sentencizer")
    raw = requests.get("https://www.gutenberg.org/files/1342/1342-0.txt").text
    clean = re.sub(r"(?s)\*\*\* START.*?END \*\*\*", "", raw)
    doc = nlp(clean)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]

    chunks, cur, cur_len, LIM = [], "", 0, 512
    for s in sentences:
        tok = s.split()
        if cur_len + len(tok) > LIM:
            chunks.append(cur.strip()); cur, cur_len = "", 0
        cur += s + " "; cur_len += len(tok)
    if cur.strip(): chunks.append(cur.strip())

    np.save(chunk_path, chunks)

print(f"Chunks loaded: {len(chunks)}")

Chunks loaded: 258


In [ ]:
from sentence_transformers import SentenceTransformer

# Load embedding model once before search
model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
embeddings = model.encode(chunks, convert_to_numpy=True)
faiss.normalize_L2(embeddings)
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)

In [ ]:
import faiss

def search_chunks(query, k=5):
    q_vec = model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q_vec)
    D, I = index.search(q_vec, k)
    return [(chunks[i], float(D[0][j])) for j, i in enumerate(I[0])]

In [ ]:
pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 42.4 MB/s eta 0:00:00


In [ ]:
!pip install llama-cpp-python --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 10.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
!wget -P /content/drive/MyDrive/ "https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf"

--2025-05-22 09:52:56--  https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.88, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/6c/c2/6cc203a4b605b1f54529984e310dd5d122bd444bc30b4123a35527d02845d4cb/9fecc3b3cd76bba89d504f29b616eedf7da85b96540e490ca5824d3f7d2776a0?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf%3B+filename%3D%22tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf%22%3B&Expires=1747911176&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzkxMTE3Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzZjL2MyLzZjYzIwM2E0YjYwNWIxZjU0NTI5OTg0ZTMxMGRkNWQxMjJiZDQ0NGJjMzBiNDEyM2EzNTUyN2QwMjg0NWQ0Y2IvOWZlY2MzYjNjZDc2YmJhODlkN

In [ ]:
from llama_cpp import Llama

llm = Llama(
    model_path="/content/drive/MyDrive/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",
    n_ctx=800,
    n_threads=2  # tune based on CPU core count
)

## This code was replaced with the other def

def answer_question(query, k=5):
    retrieved = search_chunks(query, k=k)
    context = "\n\n".join([r[0] for r in retrieved])
    
    prompt = f"""Answer the following question based only on the context.

Context:
{context}

Question: {query}
Answer:"""

    output = llm(prompt, max_tokens=512, stop=["Question:"])
    return output["choices"][0]["text"].strip()

In [ ]:
def answer_question(query, k=1):  # use fewer chunks
    retrieved = search_chunks(query, k=k)
    context = "\n\n".join([r[0] for r in retrieved])
    if len(context) > 1500:  # optional: hard truncate
        context = context[:1500]

    prompt = f"""Answer the following question based only on the context.

Context:
{context}

Question: {query}
Answer:"""

    output = llm(prompt, max_tokens=300, stop=["Question:"])
    return output["choices"][0]["text"].strip()

In [ ]:
ans = answer_question("Why does Elizabeth Bennet reject Mr. Darcy's first proposal?", k=1)
print(ans)

llama_perf_context_print:        load time =   23725.10 ms
llama_perf_context_print: prompt eval time =   23724.05 ms /   452 tokens (   52.49 ms per token,    19.05 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   23725.87 ms /   453 tokens


In [ ]:
faiss.write_index(index, "/content/drive/MyDrive/faiss_index.index")
np.save("/content/drive/MyDrive/embeddings.npy", embeddings)

In [ ]:
for chunk, score in search_chunks("Mr. Darcy", k=3):
    print(f"Score: {score:.3f}")
    print(textwrap.fill(chunk, 100))
    print("-" * 80)

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

def gradio_qa(query, k=1):
    return answer_question(query, k=k)

iface = gr.Interface(
    fn=gradio_qa,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter your question here..."),
        gr.Slider(minimum=1, maximum=5, step=1, value=1, label="Number of chunks to retrieve (k)")
    ],
    outputs="text",
    title="Gutenberg Book Q&A",
    description="Ask questions about *Pride and Prejudice* using a TinyLlama + FAISS RAG pipeline."
)

iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://41adac03cea3e5fa19.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
